In [1]:
# import matplotlib
# matplotlib.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import pandas as pd

In [ ]:
def get_data_from_file(datafilenum):
    data_file = f"../data/output{datafilenum}.csv"
    df = pd.read_csv(data_file)

    if datafilenum >= 19:
        return list(df['toxicity']), list(df['similarity']), list(df['generation'])
        
    return list(df['toxicity']), list(df['similarity'])



In [ ]:
def show_data(datafilenum):
    data = get_data_from_file(datafilenum)
    plt.xlabel("toxicity")
    plt.ylabel("similarity")

    by_generation = {}
    for i in range(len(data[2])):
        gen_id = data[2][i]
        if gen_id > 20:
            break
        if gen_id not in by_generation:
            by_generation[gen_id] = []

        by_generation[gen_id].append((data[0][i], data[1][i]))

    tot_colors = len(by_generation)
    for i, pr in enumerate(by_generation.items()):
        _, datapoints = pr
        plt.scatter([item for item, _ in datapoints], [item for _, item in datapoints], color=[i / tot_colors, i / tot_colors, i / tot_colors])
        
    plt.show()

show_data(21)
# data = get_data_from_file(20), get_data_from_file(13), get_data_from_file(16)]
# # plt.scatter(data[0][0], data[0][1], color="blue")
# # plt.scatter(data[1][0], data[1][1], color="red")

# plt.scatter(data[2][0], data[2][1], color="green")


In [ ]:
print("hi")
data = [get_data_from_file(11), get_data_from_file(13), get_data_from_file(16)]
plt.xlabel("toxicity")
plt.ylabel("similarity")
# plt.scatter(data[0][0], data[0][1], color="blue")
# plt.scatter(data[1][0], data[1][1], color="red")
plt.scatter(data[2][0], data[2][1], color="green")


plt.show()

## dataset 10, 11:
Population based search. Keep the upper frontier at all times, to generate the next sentence, find a new item at random and morph it. Total 1000 generations of this. Add % = 40

## dataset 12
same as above but add percent is 50

## dataset 13
same as above but add percent is 30

## dataset 14
The frontier is more loosely maintained, with some randomness set to 30% a suboptimal point might still be kept in the frontier. 

## dataset 16
We change the distance function to be 0/1, and the toxicity displayed here is just a measure of how many perturbations is it away from being a non-toxic prompt. 